In [2]:
%matplotlib inline

import numpy as np
import pandas as pd
import time

import matplotlib
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv("data/maintenance.csv", header=[0])

In [4]:
#
# cleanup bad data
#

df.fillna(0, inplace=True)
df["136088202.3"].replace("mean", 0, inplace=True)
df[df.columns[2:]] = df[df.columns[2:]].applymap(pd.to_numeric)

In [5]:
# standarize numeric columns
num_columns = df.columns[2:]

standarized  =  df[num_columns].sub(df[num_columns].mean(axis=0), axis=1).div(df[num_columns].std(axis=0)+1e-20, axis=1)
normalized01 = df[num_columns].sub(df[num_columns].min(axis=0), axis=1).div(df[num_columns].max(axis=0)-df[num_columns].min(axis=0)+1e-20, axis=1)
df[num_columns] = normalized01



In [6]:
#num_columns = df.columns[2:2+26]
num_columns

Index(['136088194', '136088202', '136088394', '136088802', '136089546',
       '136110468', '136216674', '136222202', '136222210', '136222234',
       ...
       '136225010.4', '136675426.4', '136675626.4', '136676578.4',
       '136676650.4', '136676666.4', '136676682.4', '136676698.4',
       '136676714.4', '136676754.4'],
      dtype='object', length=130)

In [7]:
df_y_train = pd.read_csv("data/train_label.csv", header=[0])
df_y_test = pd.read_csv("data/test_label.csv", header=[0])

y_train = df_y_train["label"].values
# remove fiest NaN element - TImeSeriesGenerator() will assemble the righ sequences for us
y_train = np.delete(y_train,0)
X_train = df[num_columns].loc[0:len(y_train)-1].values

X_test = df[num_columns].loc[len(df_y_train):].values
y_test = df_y_test["label"].apply(lambda x: 0 if x < 0.5 else 1).values

while len(X_test) < len(y_test):
    print("extending X_test to macth y_test")
    X_test = np.vstack((X_test, X_test[-1]))

len(X_train), len(y_train)
len(X_test), len(y_test)


extending X_test to macth y_test


(300, 300)

In [8]:
from sklearn import preprocessing
from sklearn.metrics import roc_curve, auc, roc_auc_score, confusion_matrix

from keras.datasets import mnist
from autokeras.image.image_supervised import ImageClassifier
from autokeras.utils import pickle_from_file
from keras.models import Sequential
from keras.layers import Dense, Reshape, Bidirectional
from keras.layers import Dropout, TimeDistributed
from keras.layers import LSTM, ConvLSTM2D, BatchNormalization, Flatten
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.callbacks import EarlyStopping

from keras_tqdm import TQDMNotebookCallback


Using TensorFlow backend.


In [9]:
# parameters 

lag = 3 # days


In [21]:
def parameter_generator():
    for lag in range(4,5, 1): # 4
        for batch_size in range(45,46,10):
            for lstm_cells in range(41,42,2): # (21, 22, 5)
                for epochs in range(200, 250, 100):
                    for steps_per_epoch in range(30, 40, 20):
                        yield (lag, batch_size, lstm_cells, epochs, steps_per_epoch)
            
for p in parameter_generator():
    print(p)

(4, 45, 41, 200, 30)


In [19]:
def get_model0(lstm_cells, lag, n_features, batch_size):

    model = Sequential()
    model.add(LSTM(lstm_cells, activation='relu', return_sequences=True, input_shape=(lag, n_features)))
    #model.add(Dense(10, activation='relu'))
    model.add(TimeDistributed(Dense(1, activation='sigmoid')))
    #model.compile(optimizer='adam', loss='mse')
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    return model


def get_model3(filters, lag, n_features, batch_size):

    input_shape = (lag, n_features, 1)
    model = Sequential()
    
    
    model.add(Reshape(input_shape + (1, ), input_shape=input_shape))

#     model.add(Bidirectional(ConvLSTM2D(filters=filters, 
#                          kernel_size=(3, 3),
#                          padding='same',
#                          return_sequences=True,
#                          input_shape=input_shape)))

#     model.add(ConvLSTM2D(filters=filters, 
#                      kernel_size=(5, 5),
#                      padding='same',
#                      return_sequences=True,
#                      activation="linear",
#                      use_bias = True,    
#                      input_shape=input_shape))
#     model.add(BatchNormalization())
    model.add(Bidirectional(ConvLSTM2D(filters=filters, 
                     kernel_size=(3, 3),
                     padding='same',
                     return_sequences=True,
                     activation="linear",
                     use_bias = True,    
                     input_shape=input_shape)))

    model.add(BatchNormalization())

    model.add(TimeDistributed(Dense(100, activation="relu")))
    model.add(Flatten())
    #odel.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # rmsprop - mse
    return model


#get_model3(8, 4, 100, 10).summary()
get_model0(8, 4, 100, 10).summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 4, 8)              3488      
_________________________________________________________________
time_distributed_4 (TimeDist (None, 4, 1)              9         
Total params: 3,497
Trainable params: 3,497
Non-trainable params: 0
_________________________________________________________________


In [12]:
# X = [10,20,30,40,50,60,70,80,90,100]
# y = [1,2,3,4,5,6,7,8,9,10]


# gen = TimeseriesGenerator(X,y, length=2, batch_size=1)

# for x in gen:
#     print(x)
    
#for x in train_generator:
#    print(x[0].shape)

In [22]:
# define dataset
dataset = X_train
n_features = dataset.shape[1]

total_samples =  dataset.shape[0]
num_train_samples = int(0.75 * total_samples)

print("Total samples: {} \nNum train samples: {} \nNum test samples: {}".format(total_samples,num_train_samples,total_samples - num_train_samples ))
print("Features: {}".format(n_features))
model = None


for (lag,batch_size, lstm_cells, epochs, steps_per_epoch) in parameter_generator():

    #output = Y_train.reshape(len(Y_train),1)

    start_time = time.time()
    
    # model 0
    #train_generator = TimeseriesGenerator(X_train, y_train, length=lag, batch_size=batch_size)
    #test_generator = TimeseriesGenerator(X_test, y_test, length=lag, batch_size=1)

    # model3
    train_generator = TimeseriesGenerator(np.expand_dims(X_train, axis=2), y_train, length=lag, batch_size=batch_size)
    test_generator = TimeseriesGenerator(np.expand_dims(X_test,axis=2), y_test, length=lag, batch_size=1)

    # define model
    model = get_model3(lstm_cells, lag, n_features, batch_size)
    print(model.summary())
    
    # early stopping criteria
    early_stopping_callback = EarlyStopping(monitor="acc",patience=10)

    # fit model
    model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch,epochs=epochs, verbose=0, 
                        callbacks=[early_stopping_callback, TQDMNotebookCallback()])


    in_sample_acc = model.evaluate_generator(train_generator)[1]
    acc = model.evaluate_generator(test_generator)[1]
    
    y_score = model.predict_generator(test_generator)
    y_true = y_test[-len(y_score):]

    fpr, tpr , thresholds = roc_curve(y_true, y_score)
    threshold = thresholds[np.argmax(tpr - fpr)]

    print("time:", time.time() - start_time)
    print("accuracy: {} in sample_acc: {} lags: {}  batch_size: {} lstm_cells: {} epochs: {} steps: {}".format(acc, in_sample_acc, lag, batch_size, lstm_cells, epochs, steps_per_epoch))
    print (calc_accuracy(model, test_generator, threshold), "AUC", roc_auc_score(y_true, y_score))
    print('threshold:', threshold)
    print ("confusion matrix \n", confusion_matrix(y_true, [1 if p >= threshold else 0 for p in y_score]))
    
print ("done")

Total samples: 683 
Num train samples: 512 
Num test samples: 171
Features: 130
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_5 (Reshape)          (None, 4, 130, 1, 1)      0         
_________________________________________________________________
bidirectional_5 (Bidirection (None, 4, 130, 1, 82)     124312    
_________________________________________________________________
batch_normalization_5 (Batch (None, 4, 130, 1, 82)     328       
_________________________________________________________________
time_distributed_6 (TimeDist (None, 4, 130, 1, 100)    8300      
_________________________________________________________________
flatten_5 (Flatten)          (None, 52000)             0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 52001     
Total params: 184,941
Trainable params: 184,777
Non-trainable 


time: 246.30467009544373
accuracy: 0.8648648648648649 in sample_acc: 1.0 lags: 4  batch_size: 45 lstm_cells: 41 epochs: 200 steps: 30
(36.36363636363637, 95.62043795620438, 91.21621621621621) AUC 0.6289814200398143
threshold: 0.97555
confusion matrix 
 [[262  12]
 [ 14   8]]
done


In [ ]:

fpr, tpr , thresholds = roc_curve(y_true, y_score)
t = thresholds[np.argmax(tpr - fpr)]
print(t)
print(confusion_matrix(y_true, [1 if p >= t else 0 for p in y_score]))
print(confusion_matrix(y_true, [1 if p >= 0.5 else 0 for p in y_score]))
plot_auc(y_true, y_score)



In [14]:
def calc_accuracy(model, test_generator, threshold):

    y_score = model.predict_generator(test_generator)

    i = 0
    correct1 = 0
    total1 = 0
    for x in test_generator:
        p = y_score[i]
        if (x[1]==1):
            if p >= threshold:
                correct1 += 1
                #print(i,x[0].shape, x[1], p , "*")
            #else:
                #print(i,x[0].shape, x[1], p )
            total1 += 1
        i+=1


    #print(correct1, total1)    
    #print("Accuracy (1) {:6.2f}%".format(100.0* correct1/total1))

    i = 0
    correct0 = 0
    total0 = 0
    for x in test_generator:
        p = y_score[i]
        if (x[1]==0):
            if p < threshold:
                correct0 += 1
            total0 += 1
        i+=1


    #print(correct0, total0)    
    #print("Accuracy (0) {:6.2f}%".format(100.0* correct0/total0))

    #print()
    #print("Accuracy (0+1) {:6.2f}%".format(100.0* (correct0+correct1)/(total0+total1)))
    
    return (100.0* correct1/total1, 100.0* correct0/total0, 100.0* (correct0+correct1)/(total0+total1))



In [15]:
def plot_auc(y_true, y_score):
    fpr_rf, tpr_rf, _ = roc_curve(y_true, y_score)

    plt.figure(1)
    plt.plot(fpr_rf, tpr_rf)
    plt.show()

# Using KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3)

X = []
y = []

train_generator = TimeseriesGenerator(X_train, y_train, length=4, batch_size=1)
for xy in train_generator:
    n = xy[0].shape[1] * xy[0].shape[2]
    X.append(xy[0].reshape(n))
    y.append(xy[1][0])
    
knn.fit(X, y) 

Xt = []
yt = []

test_generator = TimeseriesGenerator(X_test, y_test, length=4, batch_size=1)
for xy in test_generator:
    n = xy[0].shape[1] * xy[0].shape[2]
    Xt.append(xy[0].reshape(n))
    yt.append(xy[1][0])
    
acc = knn.score(Xt, yt) 
auc = roc_auc_score(yt, knn.predict(Xt))

print("Acc", acc, "AUC", auc)

In [ ]:
y_hat = knn.predict(Xt)
[i for i in range(0, len(y_hat)) if y_hat[i] == 1 ]


In [ ]:
[i for i in range(0, len(yt)) if yt[i] == 1 ]

In [ ]:
a = [1,2,3,4,5,6]
a[2:-1]
